In [7]:
!pip install -q langchain-openai langchain-community langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
import os
from langchain_core.messages import HumanMessage
import requests

In [9]:
@tool
def multiply(a: float, b: float) -> float:
  """Multiply two numbers a and b"""
  return a*b

In [ ]:
llm = ChatOpenAI(model_name="gpt-5")
llm_with_tools = llm.bind_tools([multiply])

In [32]:
query = HumanMessage("Multiply 2 and 3?")
messages = [query]
result = llm_with_tools.invoke(messages)

In [33]:
messages.append(result)

In [34]:
tool_result = multiply.invoke(result.tool_calls[0])

In [35]:
messages.append(tool_result)

In [37]:
messages

[HumanMessage(content='Multiply 2 and 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 218, 'prompt_tokens': 135, 'total_tokens': 353, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5SdFnyYMhLnUt3Qdt4vzaxhqqgJo', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c27c9-f337-7cb1-ac45-3a834486ae40-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_FHK5VHb7va0BF5cMokUiQY4b', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 135, 'output_tokens': 218, 'total_tokens': 353, 'input_token_details': {'audio': 0, 'cache_read'

In [36]:
llm_with_tools.invoke(messages)

AIMessage(content='6', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 168, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5SdOT4jRCAXdLC61lWouLlp3NRk6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c27ca-16f6-7c30-9f47-7a716a65bb41-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 168, 'output_tokens': 4, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Currency conversion tool

In [23]:
@tool
def currency_conversion(base_currency: str,target_currency: str) -> float:
  """Converts the base currecny to the target currency."""
  url = f'https://v6.exchangerate-api.com/v6/9b9f768fb5ea98f7aca29fbc/pair/{base_currency}/{target_currency}'
  response = requests.get(url)
  data = response.json()
  return data

@tool
def convert(base_currency_value: int, conversion_rate: float) -> float:
  """Given a conversion rate this function converts it to the target currency by multiplying it with base currency value"""
  return base_currency_value * conversion_rate

In [24]:
llm = ChatOpenAI(model_name="gpt-5")
llm_with_tools = llm.bind_tools([multiply, currency_conversion, convert])

In [43]:
query = HumanMessage("I currently have 500000 indian rupees. How much is it in dollars")
messages = [query]
result = llm_with_tools.invoke(messages)
result

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 286, 'prompt_tokens': 215, 'total_tokens': 501, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5VqrGOsTMbIISOpXK9OsypsiqKCY', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2886-cdc3-7fd0-998d-7e8e6a2e45d0-0', tool_calls=[{'name': 'currency_conversion', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'call_cTZR04OJyTyosoXXhMvfSUs4', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 215, 'output_tokens': 286, 'total_tokens': 501, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, '

In [44]:
messages.append(result)

In [32]:
result

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 291, 'prompt_tokens': 215, 'total_tokens': 506, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5VnS2k7ceyNUTGQiQk7DRQNLQv72', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2883-9456-7df3-bccb-f441959ecf5c-0', tool_calls=[{'name': 'currency_conversion', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'call_mWblCjIILOezfqVDyUi2qkCM', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 215, 'output_tokens': 291, 'total_tokens': 506, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, '

In [45]:
tool_message = currency_conversion.invoke(result.tool_calls[0])

In [46]:
messages.append(tool_message)

In [47]:
message = llm_with_tools.invoke(messages)

In [48]:
messages.append(message)

In [49]:
message

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 390, 'total_tokens': 417, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5VrTn2nQvkE3dXFQqzKNw1klgQQd', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2887-63f6-7ec3-a82f-6ea1fa33988f-0', tool_calls=[{'name': 'convert', 'args': {'base_currency_value': 500000, 'conversion_rate': 0.01107}, 'id': 'call_P5zYYguNfrh2S6UGTFTOzNRK', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 390, 'output_tokens': 27, 'total_tokens': 417, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoni

In [51]:
messages.append(convert.invoke(message.tool_calls[0]))

In [53]:
messages

[HumanMessage(content='I currently have 500000 indian rupees. How much is it in dollars', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 286, 'prompt_tokens': 215, 'total_tokens': 501, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5VqrGOsTMbIISOpXK9OsypsiqKCY', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2886-cdc3-7fd0-998d-7e8e6a2e45d0-0', tool_calls=[{'name': 'currency_conversion', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'call_cTZR04OJyTyosoXXhMvfSUs4', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 215,

In [52]:
llm_with_tools.invoke(messages)

AIMessage(content='500,000 INR is approximately 5,535 USD at the current rate (~0.01107 USD per INR). Rates fluctuate, so check a live source if you need an exact amount for a transaction.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 431, 'total_tokens': 476, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5VsKWoxyz5S9V60jRduzFSNlnsDy', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2888-2e2d-7e53-a564-a5f9bfeae33b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 431, 'output_tokens': 45, 'total_tokens': 476, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audi